# Atividade 4 - PO-202
Grupo:

Gabriel Telles Missailidis (ITA)

Rafael Silva de Oliveira (ITA)

Samir Nunes da Silva (ITA)

In [248]:
from pulp import *


# Problema 1: Onde instalar bases do SAMU

Uma cidade precisa decidir onde instalar bases do SAMU atendendo todas as regiões da cidade em até 15 minutos. Existem cinco possíveis locais de instalação. O local 1 atende as regiões sul e central em até 15 minutos. O local 2 atende as regiões sul, central e sudeste. O local 3 atende as regiões oeste e central em até 15 minutos. O local 4 atende as regiões central, norte e oeste em até 15 minutos. O local 5 atende as regiões oeste,  norte e sudeste em até 15 minutos.  Deseja-se minimizar o número de bases instaladas. Modele como um problema de programação linear inteira e resolva o PuLP.

In [249]:
prob1 = LpProblem('P1', LpMinimize)


In [250]:
x1 = pulp.LpVariable('x1', lowBound=0, cat='Integer')  # Base do local 1
x2 = pulp.LpVariable('x2', lowBound=0, cat='Integer')  # Base do local 2
x3 = pulp.LpVariable('x3', lowBound=0, cat='Integer')  # Base do local 3
x4 = pulp.LpVariable('x4', lowBound=0, cat='Integer')  # Base do local 4
x5 = pulp.LpVariable('x5', lowBound=0, cat='Integer')  # Base do local 5


In [251]:
z = x1 + x2 + x3 + x4 + x5
prob1 += z


In [252]:
rest1 = x1 + x2  # região sul
rest2 = x1 + x2 + x3 + x4  # região central
rest3 = x2 + x5  # região sudeste
rest4 = x4 + x5  # região norte
rest5 = x3 + x4 + x5  # região oeste

prob1 += (rest1 >= 1)
prob1 += (rest2 >= 1)
prob1 += (rest3 >= 1)
prob1 += (rest4 >= 1)
prob1 += (rest5 >= 1)


In [253]:
prob1


P1:
MINIMIZE
1*x1 + 1*x2 + 1*x3 + 1*x4 + 1*x5 + 0
SUBJECT TO
_C1: x1 + x2 >= 1

_C2: x1 + x2 + x3 + x4 >= 1

_C3: x2 + x5 >= 1

_C4: x4 + x5 >= 1

_C5: x3 + x4 + x5 >= 1

VARIABLES
0 <= x1 Integer
0 <= x2 Integer
0 <= x3 Integer
0 <= x4 Integer
0 <= x5 Integer

In [254]:
optimization_result = prob1.solve()


In [255]:
assert optimization_result == LpStatusOptimal


In [256]:
for var in [x1, x2, x3, x4, x5]:
    print(f'{var.name}:{var.value()}')


x1:1.0
x2:0.0
x3:0.0
x4:0.0
x5:1.0


# Problema 2: Organizando o transporte corporativo
Considere uma empresa que possui 402 empregados, distrbuídos em 30 pontos na cidade, e precisa organizar o transporte de ida e volta destes empregados e forma a minimizar seu custo mensal. A empresa pode alugar até 10 ônibus com 48 lugares, doravante ônibus 1, e até 30 micro-ônibus com 16 lugares, doravante ônibus 2. O ônibus 1 tem custo fixo mensal de R$15000,00 e o 2 de R$4000,00. O custo por km rodado do õnibus 1 é de R$3,20 e do ônibus 2 é R$2,90. Os demais dados seguem no arquivo dat.txt. Os empregados vão e voltam no mesmo ônibus e do mesmo ponto.A empresa precisa de uma solução, de preferência ótima, em até 1 minuto. Proponha uma solução para a empresa utilizando otimização.

### Matriz de distâncias

In [257]:
import csv

Manualmente, preparou-se previamente os dados de forma a considerar a a empresa como ponto 0. Além disso, foram adicionados as distâncias entre a empresa e os pontos, de forma a obter uma matriz 31 x 31, com as relações entre os pontos 0 (empresa), 1, 2, ..., 30.

In [258]:
with open('mat.csv', newline='') as f:
    reader = csv.reader(f, delimiter=';')
    mat_distancias = list(reader)

# convertendo os elementos da matriz de str para float
for i in range(0, 31):
   for j in range(0, 31):
        mat_distancias[i][j] = float(mat_distancias[i][j])



### Número de empregados por ponto

In [259]:
numero_empregados = [
    0,  # Ponto 0, empresa
    14, # Ponto 1
    13, # Ponto 2
    15, # ...
    10,
    10,
    15,
    5,
    12,
    15,
    16,
    12,
    6,
    13,
    18,
    15,
    18,
    17,
    9,
    16,
    16,
    15,
    13,
    11,
    15,
    14,
    12,
    11,
    18,
    16,
    12
]


### Custos dos ônibus

In [260]:
custo_km = ([3.2] * 10 + [2.9] * 30)

### Capacidade dos ônibus

In [261]:
capacidade_onibus = ([48] * 10 + [16] * 30)

### Custos fixos associados

In [262]:
custos_fixos = ([15000] * 10 + [4000] * 30)

### Definição dos Pontos e Ônibus

In [263]:
pontos = list(range(0, 31))
onibus = list(range(0, 40))

### Criação das variáveis de caminhos e rotas

In [264]:
Caminhos = [(i, j) for i in pontos for j in pontos]

In [265]:
Rotas = [(i, j, k) for i in pontos for j in pontos for k in onibus]

### Definindo a variável a ser otimizada

In [266]:
entrega = LpVariable.dicts("Entrega", (pontos, pontos, onibus), 0, 1, LpInteger)

### Definindo o problema de otimização

In [267]:
prob2 = LpProblem("P2", LpMinimize)

### Adicionando a função objetivo

In [268]:
prob2 += (
    lpSum([entrega[i][j][k] * custo_km[k] * mat_distancias[i][j] for (i, j, k) in Rotas]) 
                                +        
    lpSum([entrega[0][j][k] * custos_fixos[k]/44 for j in pontos[1:]] for k in onibus), # divisão por 22 garante que o valor é diário
    "Soma_dos_custos_de_transporte"
)

### Adicionando restrições

Pontos de parada

In [269]:
for i in pontos[1:]: # exceto o ponto 0, que é a fábrica
    prob2 += (
        lpSum([entrega[i][j][k] for j in pontos for k in onibus]) == 1,
        "Entregas_a_partir_do_ponto_%s" % i,
    )

for j in pontos[1:]: # exceto o ponto 0, que é a fábrica
    prob2 += (
        lpSum([entrega[i][j][k] for i in pontos for k in onibus]) == 1,
        "Entregas_ate_o_ponto_%s" % j,
    )

Fábrica

In [270]:
for k in onibus:
    prob2 += (
        lpSum([entrega[0][j][k] for j in pontos]) <= 1,
        "Saindo_da_fabrica_onibus_%s" % k,
    )
    prob2 += (
        lpSum([entrega[i][0][k] for i in pontos]) <= 1,
        "Chegando_na_fabrica_onibus_%s" % k,
    )

### Restrições relacionadas à "entrega" de empregados nos pontos

In [271]:
for k in onibus:
    prob2 += (
        lpSum([entrega[i][j][k] * numero_empregados[j] for (i, j) in Caminhos]) <= capacidade_onibus[k],
        "Saindo_do_ponto_onibus_%s" % k,
    )

    prob2 += (
        lpSum([entrega[i][j][k] * numero_empregados[i] for (i, j) in Caminhos]) == lpSum([entrega[i][j][k] * numero_empregados[j] for (i, j) in Caminhos]),
        "Chegando_no_ponto_onibus_%s" % k,
    )

    # Aqui, começamos a partir do índice 1 para desconsiderar a empresa, que é o ponto inicial
    prob2 += (
        lpSum([entrega[i][j][k] + entrega[j][i][k] for i in pontos[1:] for j in pontos[1:]]) <= 1,
        "Unicidade_dos_caminhos_do_onibus_%s" % k,
    )

### Resolvendo o problema

In [ ]:
prob2.solve()

print("Status:", LpStatus[prob2.status])

### Matriz de utilização dos ônibus

In [ ]:
for k in onibus:
    print(k," = ",lpSum([entrega[i][j][k].varValue for (i, j) in Caminhos]))

### Finalmente, para o custo total:

In [ ]:
print("Custo total dos Transportes = R${:.2f}".format(value(prob2.objective)))

In [ ]:
# imprimir 40 matrizes 31 x 31
# for k in onibus:
#     print([entrega[i][j][k].varValue for (i,j) in Caminhos])

# Problema 3: O Problema da Galeria de Arte

Considere uma galeria de arte com muitos corredores e curvas. A galeria está exibindo pinturas muito valiosas e você deseja mantê-las seguras. Você está planejando instalar câmeras de segurança em cada corredor para que as câmeras tenham todas as pinturas à vista. Se houver uma câmera de segurança em um corredor, ela poderá ver todas as pinturas do corredor. Se houver uma câmera no canto onde dois corredores se encontram (o turno), ela pode ver pinturas em ambos. Podemos modelar este sistema como um grafo onde os nós representam os lugares onde os corredores se encontram ou quando um corredor se torna um beco sem saída, e as arestas são os corredores. Considere o exemplo abaixo. Modele o problema como PPI e resolva usando o PuLP.

In [ ]:
prob3 = LpProblem('P3', LpMinimize)


In [ ]:
x1 = LpVariable('x1', lowBound=0, cat='Integer')
x2 = LpVariable('x2', lowBound=0, cat='Integer')
x3 = LpVariable('x3', lowBound=0, cat='Integer')
x4 = LpVariable('x4', lowBound=0, cat='Integer')
x5 = LpVariable('x5', lowBound=0, cat='Integer')
x6 = LpVariable('x6', lowBound=0, cat='Integer')
x7 = LpVariable('x7', lowBound=0, cat='Integer')
x8 = LpVariable('x8', lowBound=0, cat='Integer')
x1_2 = LpVariable('x1_2', lowBound=0, cat='Integer')
x2_3 = LpVariable('x2_3', lowBound=0, cat='Integer')
x3_4 = LpVariable('x3_4', lowBound=0, cat='Integer')
x4_5 = LpVariable('x4_5', lowBound=0, cat='Integer')
x5_6 = LpVariable('x5_6', lowBound=0, cat='Integer')
x6_7 = LpVariable('x6_7', lowBound=0, cat='Integer')
x7_8 = LpVariable('x7_8', lowBound=0, cat='Integer')
x1_8 = LpVariable('x1_8', lowBound=0, cat='Integer')
x5_8 = LpVariable('x5_8', lowBound=0, cat='Integer')
x2_5 = LpVariable('x2_5', lowBound=0, cat='Integer')


In [ ]:
z = x1 + x2 + x3 + x4 + x5 + x6 + x7 + x1_2 + x2_3 + \
    x3_4 + x4_5 + x5_6 + x6_7 + x7_8 + x1_8 + x5_8 + x2_5
prob3 += z


In [ ]:
rest1 = x1_2 + x1 + x2
rest2 = x2_3 + x2 + x3
rest3 = x3_4 + x3 + x4
rest4 = x4_5 + x4 + x5
rest5 = x5_6 + x5 + x6
rest6 = x6_7 + x6 + x7
rest7 = x7_8 + x7 + x8
rest8 = x1_8 + x1 + x8
rest9 = x5_8 + x5 + x8
rest10 = x2_5 + x2 + x5

prob3 += (rest1 >= 1)
prob3 += (rest2 >= 1)
prob3 += (rest3 >= 1)
prob3 += (rest4 >= 1)
prob3 += (rest5 >= 1)
prob3 += (rest6 >= 1)
prob3 += (rest7 >= 1)
prob3 += (rest8 >= 1)
prob3 += (rest9 >= 1)
prob3 += (rest10 >= 1)


In [ ]:
prob3


In [ ]:
optimization_result = prob3.solve()


In [ ]:
assert optimization_result == LpStatusOptimal


In [ ]:
var_list = [x1, x2, x3, x4, x5, x6, x7, x8, x1_2, x2_3,
            x3_4, x4_5, x5_6, x6_7, x7_8, x1_8, x5_8, x2_5]
for var in var_list:
    print(f'{var.name}:{var.value()}')


In [ ]:
s = 0
for var in var_list:
    s = s + var.value()
print(s)
